<a href="https://colab.research.google.com/github/Solajungq/ta_indiv/blob/main/TA_Individual_Assignment_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment effect of artificial intelligence

# build a full RAG pipeline using Llama Index

We will install an open-source, smallish LLM Phi-3, from Microsoft, to run locally on our Colab instance to do the generation. To augment the results we will give the LLM access to our ArXiv abstracts this time stored in Chroma db as a vector store/document store. Finally, we use Ollama as an interface to interact with Phi-3 on our machine.

## Install Ollama

In [ ]:
# Install Ollama v0.1.30
!curl https://ollama.ai/install.sh | sed 's#https://ollama.ai/download#https://github.com/jmorganca/ollama/releases/download/v0.1.30#' | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Downloading ollama...
100 10091    0 10091    0     0  31253      0 --:--:-- --:--:-- --:--:-- 31241
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


Next we do a bit of work to get Ollama running in the background of our Colab (Linux) instance

In [ ]:
# Setup the model as a global variable
OLLAMA_MODEL='phi:latest'

# Add the model to the environment of the operating system
import os
os.environ['OLLAMA_MODEL'] = OLLAMA_MODEL # declare this as a global variable (our model)
!echo $OLLAMA_MODEL # print the global variable to check it saved

import subprocess
import time

# Start ollama on the server ("serve")
command = "nohup ollama serve&" # "nohup" and "&" means run in the background

# Use subprocess.Popen to run the command
process = subprocess.Popen(command,
                            shell=True,
                            stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE)

print("Process ID:", process.pid) # print the process ID
time.sleep(5)  # Makes Python wait for 5 seconds

!ollama -v # print the Ollama version number as a check

phi:latest
Process ID: 1742
ollama version is 0.1.38


## Pull the model

In [ ]:
# Query the model via the command line
# First time running it will "pull" (import) the model
!ollama run $OLLAMA_MODEL "Give me short summary of the environmental effect of artificial intelligence"

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠧ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pu

## Install packages

In [ ]:
# Install prerequisites
!pip install llama-index-embeddings-huggingface
!pip install llama-index-llms-ollama
!pip install llama-index-vector-stores-chroma
!pip install llama-index ipywidgets
!pip install llama-index-llms-huggingface
!pip install chromadb

# Import required modules from the llama_index library
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.core import StorageContext

# Import ChromaVectorStore and chromadb module
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb

# Import the Ollama class
from llama_index.llms.ollama import Ollama

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Usin

## Connect LLM (Ollama)

In [ ]:
# Use the global variable (OLLAMA_MODEL) as our LLM
# Set a timeout of 4 minutes
llm = Ollama(model=OLLAMA_MODEL, request_timeout=240.0)

## Prompt template

In [ ]:
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate

qa_prompt_str = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the question: {query_str}\n"
)

# Text QA Prompt
chat_text_qa_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=(
            "Always answer the question, even if the context isn't helpful."
        ),
    ),
    ChatMessage(role=MessageRole.USER, content=qa_prompt_str),
]

text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

# Augment with Arxiv

In [ ]:
!mkdir -p '/content/data/' # create an empty directory called "data"
!pip install arxiv

import arxiv

# Number of records
n_records = 10

# Construct the default API client.
client = arxiv.Client()

# Search for the n most recent articles matching the keyword "text analytics."
search = arxiv.Search(
  query = "environmental effect of the artificial intelligence", # search query
  max_results = n_records, # number of records to return - defined above
  sort_by = arxiv.SortCriterion.SubmittedDate # sort order (submission date)
)

results = client.results(search) # collect results

count = 0

for r in client.results(search): # iterate through the results
  doc = r.summary
  # create a file name as a combination of the path ("/content/data/") ...
  # the name "Ouput", a number that increases with each abstract (0-499) ...
  # and the file type (".txt").
  # e.g. the first file will be "/content/data/Output0.txt" ...
  # and the last will be "/content/data/Output499.txt"
  fname = "/content/data/Output" + str(count) + ".txt"
  with open(fname, "w") as text_file:
    text_file.write(doc) # save the file
  count += 1 # increment the count

UnexpectedEmptyPageError: Page of results was unexpectedly empty (https://export.arxiv.org/api/query?search_query=environmental+effect+of+the+artificial+intelligence&id_list=&sortBy=submittedDate&sortOrder=descending&start=400&max_results=100)

In [ ]:
# Check the first one to make sure everything worked:
first_file = open('/content/data/Output0.txt', 'r')
file_contents = first_file.read() # read the document
print(file_contents)
first_file.close() # close when finished

In [ ]:
# Use Llama Index's SimpleDirectoryReader to load all the document
# Load documents
reader = SimpleDirectoryReader("/content/data") # load documents from the /data folder
docs = reader.load_data()
print(f"Loaded {len(docs)} docs")

# Initialize a HuggingFace Embedding model
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

# Specify the LLM and embedding model into LlamaIndex's settings
Settings.llm = llm
Settings.embed_model = embed_model

## Build Vector database

In [ ]:
# Create client ("db") and a database ("chroma_db")
db = chromadb.PersistentClient(path="./chroma_db") # chromadb: should be downloaded. Vector DB

# Create a collection/table ("demo-for-ram") in the db
chroma_collection = db.create_collection("demo-for-ram")

# Set up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
# Specify Chroma as our vector db
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create the index
index = VectorStoreIndex.from_documents(
    docs, # the abstract documents (e.g. "Output0.txt")
    storage_context = storage_context,
    embed_model = embed_model
)

# Print the metadata
print(chroma_collection)

# Print the name of the collection (table)
print(f'Collection name is: {chroma_collection.name}')

# Query to RAG

In [ ]:
query_engine = index.as_query_engine() # use the vector db for queries

response = query_engine.query("What are LLMs?") # query Phi-3 with context
response.response # print the response

In [ ]:
response.metadata # print the full output from the LLM